In [3]:
import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [4]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

def build_model(data):

    print(data.shape)
    # this is the size of our encoded representations
    encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

    # this is our input placeholder
    original_dim=data.shape[1]
    input_img = Input(shape=(original_dim,))
    # "encoded" is the encoded representation of the input

    encoded = Dense(256, activation='tanh',
                    activity_regularizer=regularizers.l1(2*10e-5))(input_img)
    encoded = Dense(128, activation='tanh',
                    activity_regularizer=regularizers.l1(2*10e-5))(encoded)
    encoded = Dense(32, activation='tanh',
                    activity_regularizer=regularizers.l1(2*10e-5))(encoded)

    decoded = Dense(128, activation='tanh')(encoded)
    decoded = Dense(256, activation='tanh')(decoded)
    decoded = Dense(original_dim, activation='tanh')(decoded)

    # this model maps an input to its reconstruction
    autoencoder = Model(input_img, decoded)

    # this model maps an input to its encoded representation
    encoder = Model(input_img, encoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    autoencoder.fit(data, data,
                    epochs=10,
                    batch_size=100,
                    shuffle=True)
    return encoder

Using TensorFlow backend.


In [5]:
from nilearn.decomposition import CanICA
def prepare_data(func_filenames):
    canica = CanICA(memory="nilearn_cache", memory_level=2,
                    threshold=3., verbose=10, random_state=0, 
                    mask='/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/ADHD200_mask_152_4mm.nii.gz')
    data=canica.prepare_data(func_filenames)
    return data

In [6]:
from nilearn.connectome import ConnectivityMeasure

def corr(all_time_series):
    connectivity_biomarkers = {}
    conn_measure = ConnectivityMeasure(kind='correlation', vectorize=True)
    connectivity_biomarkers = conn_measure.fit_transform(all_time_series)
    return connectivity_biomarkers

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def classify(train_X,train_Y, test_X, test_Y):
    names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
             "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
             "Naive Bayes", "QDA"]

    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        GaussianProcessClassifier(1.0 * RBF(1.0)),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        MLPClassifier(alpha=1),
        AdaBoostClassifier(),
        GaussianNB(),
        QuadraticDiscriminantAnalysis()]

    scores = []
    
    for name, clf in zip(names, classifiers):
        clf.fit(train_X, train_Y) 
        score=clf.score(test_X,test_Y)
        scores.append(score)
    return scores



    

In [8]:
#X=np.load('/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/NYU/All_Data_20.npy')
print("loading")
X=np.load('/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/NYU/All_Data.npy')
Y=np.load('/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/NYU/All_Labels.npy')

loading


In [12]:
T=172

In [ ]:
from sklearn.model_selection import StratifiedKFold


    
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

# trick. split(Y,Y) instead of split(X,Y) due to X is already concatnated 
for train,test in kfold.split(Y,Y):   
    #indexing of specific subjects
    print(test)
    train_X = [X[i*T:i*T+T] for i in train]    

    train_Y = [Y[i] for i in train]
    test_Y = [Y[i] for i in test]

    #concat all subjects
    model=build_model(np.vstack(train_X))

    print("Computing Correlation")
    train_D = [model.predict(X[i*T:i*T+T]) for i in train]
    test_D = [model.predict(X[i*T:i*T+T]) for i in test]

    #Release GPU memory after model is used
    from keras import backend as K
    K.clear_session()

    train_FC=corr(train_D)
    test_FC=corr(test_D)

    score=classify(train_FC,train_Y, test_FC, test_Y)
    cvscores.append(score)


[ 11  21  25  31  32  34  35  36  41  42  50  52  55  63  70  75  85  90
 100 107 109 116 120 125 132 135 139 141 147 148 151 159 166 169 172 184
 189 191 194 197 208 210 211 215]


In [ ]:
cvscores